In [1]:
using JuMP, Gurobi, Random, LinearAlgebra, Plots, PyCall

In [2]:
pushfirst!(PyVector(pyimport("sys")."path"), "")
init  = pyimport("__init__")

PyObject <module '__init__' from '/home/harcenage/Documents/CLSPM/julia/__init__.py'>

In [3]:
mutable struct solution
    x :: Matrix{Float64}
    I :: Matrix{Float64}
    y :: Matrix{Int64}
    z :: Vector{Int}
    c :: Vector{Float64}
    obj :: Float64
    fitness :: Float64
end 

In [4]:
function model(instance_dict, pl = false)
    P = instance_dict["P"]
    T = instance_dict["T"]
    len_T = length(T)
    set_up_cost = instance_dict["set_up_cost"]
    variable_prod_cost = instance_dict["variable_prod_cost"]
    holding_cost = instance_dict["holding_cost"]
    mtn_cost = instance_dict["mtn_cost"]
    demand = instance_dict["demand"]
    alpha = instance_dict["alpha"]
    cmax = instance_dict["cmax"]
    
    model = Model(Gurobi.Optimizer)
    @variable(model, 0 <= x[P,T])
    @variable(model, 0 <= I[P,T])
    @variable(model, 0 <= c[T])
    if pl
        @variable(model, 0 <= y[P,T] <= 1)
        @variable(model, 0 <= z[T] <=1)
    else
        @variable(model, 0 <= y[P,T] <= 1, Bin)
        @variable(model, 0 <= z[T] <=1, Bin)
    end
    
    @constraint(model, c1[i in P], x[i,1] - I[i,1] == demand[i,1])
    
    @constraint(model, c2[i in P, t in 2:len_T], x[i,t] + I[i,t-1] - I[i,t] == demand[i,t])
    
    @constraint(model, c3[i in P, t in T], x[i,t] <= (sum(demand[i,s] for s in t:len_T))*y[i,t])
    
    @constraint(model, c4[t in T], sum(x[i,t] for i in P) <= c[t])
    
    @constraint(model, c5[t in T], c[t] <= cmax )
    
    @constraint(model, c6[t in 2:len_T], c[t] <= alpha*c[t-1] + cmax*z[t])
    
    @objective(model, Min, sum(set_up_cost[i,t]*y[i,t] + variable_prod_cost[i,t]*x[i,t] + holding_cost[i,t]*I[i,t] for i in P, t in T) + sum(mtn_cost[t]*z[t] for t in T))
    
    JuMP.optimize!(model)
    sx = JuMP.value.(x)
    sI = JuMP.value.(I)
    sy = JuMP.value.(y)
    sz = JuMP.value.(z)
    sc = JuMP.value.(c)
     #=
    for i in P
        println(sum(sy[i,:]))
    end
    =# 
    nbr_set_up = []
    println(sum(sz))
    println(sum(sz))
    for i in P
        push!(nbr_set_up, sum(sy[i,:]))
    end
    println(nbr_set_up)
    return sz, sc, sy, sx, sI
end

model (generic function with 2 methods)

In [1]:
function construct_capacities(z, t, alpha, cmax)
    c = zeros(Float64,t)
    for j in 1:t
        if z[j] ==1
            c[j] = cmax
        else
            c[j] = alpha*c[j-1]
        end
    end
    return c      
end 

function pop_initial(len_pop,instance_dict)
    P = instance_dict["P"]
    T = instance_dict["T"]
    p = length(P)
    t = length(T)
    alpha = instance_dict["alpha"]
    cmax = instance_dict["cmax"]
    variable_prod_cost = instance_dict["variable_prod_cost"]
    demand = instance_dict["demand"]
    holding_cost = instance_dict["holding_cost"]
    mtn_cost = instance_dict["mtn_cost"]
    set_up_cost = instance_dict["set_up_cost"]
    println(P)
    
    unf_cost = sum(variable_prod_cost[i,j]*demand[i,j] + holding_cost[i,j]*demand[i,j] + set_up_cost[i,j] for i in P, j in T) + sum(mtn_cost[j] for j in T)
    
    nbr_mtn = 0
    capacity = cmax
    for j in 2:t
        temp = (sum(demand))/t
        if alpha*capacity < temp
            nbr_mtn += 1
            capacity = cmax
        else
            capacity *= alpha
        end
    end
    
    #println(nbr_mtn)
    pop_initial_list = []
    compt = 0
    while compt < len_pop
        x = zeros(Float64,p,t)
        I = zeros(Float64,p,t)
        z = zeros(Int64,t)
        z[1] = 1 
        indices_ones = randperm(t-1)[1:nbr_mtn] .+ 1
        z[indices_ones] = ones(nbr_mtn)
        
        c = construct_capacities(z,t,alpha,cmax)
        y = zeros(Int64,p,t)
        
        y[:,1] = [1 for _ in P]
        for i in 1:p
            m = floor(Int64, t*0.9)
            indices_ones = randperm(t-1)[1:m] .+ 1
            y[i,indices_ones] = ones(m)    
        end

        #println(z)
        #println(c)
        #display(y)
        sol = solution(x,I,y,z,c,0,0);
        feasible, sx, sI, obj = find_x_I(sol.y, sol.c, instance_dict)
        #println(feasible)
        if feasible
            compt+=1
            sol.x = sx
            sol.I = sI
            sol.obj = obj + sum(set_up_cost[i,j]*y[i,j] for i in P, j in T) + sum(mtn_cost[j]*z[j] for j in T)
            push!(pop_initial_list,sol)
        else 
            compt += 1
            sol.obj = obj + unf_cost
        end
        #println(sol.obj) 
    end
    #println(compt)
    return pop_initial_list
end 


pop_initial (generic function with 1 method)

In [1]:
function verify_feasibility(solution, instance_dict)
    P = instance_dict["P"]
    T = instance_dict["T"]
    demand = instance_dict["demand"]
    c = solution.c
    y = solution.y
    list_d = []
    list_c = []
    #=
    if sum(demand) > sum(solution.c)
        return 0
    end
    =#
    for t in T
        temp = sum(demand[i,t]*y[i,t] for i in P)
        push!(list_d,temp)
        push!(list_c, solution.c[t])
        if sum(list_d) > sum(list_c)
            return 0
        end
    end
    return 1
end

function resolve_CLSP(sy,sc,instance_dict)
    P = instance_dict["P"]
    T = instance_dict["T"]
    len_T = length(T)
    variable_prod_cost = instance_dict["variable_prod_cost"]
    holding_cost = instance_dict["holding_cost"]
    demand = instance_dict["demand"]
    
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)


    @variable(model, 0 <= x[P,T])
    
    @variable(model, 0 <= I[P,T])
    
    @constraint(model, c1[i in P], x[i,1] - I[i,1] == demand[i,1])
    
    @constraint(model, c2[i in P, t in 2:len_T], x[i,t] + I[i,t-1] - I[i,t] == demand[i,t])
    
    @constraint(model, c3[i in P, t in T], x[i,t] <= (sum(demand[i,s] for s in t:len_T))*sy[i,t])
    
    @constraint(model, c4[t in T], sum(x[i,t] for i in P) <= sc[t])
    
    @objective(model, Min, sum(variable_prod_cost[i,t]*x[i,t] + holding_cost[i,t]*I[i,t] for i in P, t in T))
    #print(model)
    set_silent(model)
    JuMP.optimize!(model)
    if termination_status(model) == MOI.INFEASIBLE
        return false, 0, 0, 0
    else
        sx = JuMP.value.(x)
        sI = JuMP.value.(I)
        obj = objective_value(model)
        return true, sx, sI, obj

    end
end


function cross(parent1, parent2, list_pts, nbr_pts, t)
    #println(list_pts)
    list_parents_z = [copy(parent1), copy(parent2)]
    z = list_parents_z[1]
    ind = 2
    current = list_parents_z[ind]
    for i in 1:nbr_pts-1
        z[list_pts[i]+1: list_pts[i+1]] = current[list_pts[i]+1:list_pts[i+1]]
        if ind == 2 
            ind = 1
        else 
            ind = 2
        end
        current = list_parents_z[ind]
    end
    z[list_pts[end]+1:t] = current[list_pts[end]+1:t]
    return z
end

function crossover(sol1, sol2, nbr_pts, instance_dict)
    P = instance_dict["P"]
    T = instance_dict["T"]
    p = length(P)
    t = length(T)
    alpha = instance_dict["alpha"]
    cmax = instance_dict["cmax"]
    demand = instance_dict["demand"]
    variable_prod_cost = instance_dict["variable_prod_cost"]
    holding_cost = instance_dict["holding_cost"]
    mtn_cost = instance_dict["mtn_cost"]
    set_up_cost = instance_dict["set_up_cost"]
    parent1_z = copy(sol1.z)
    parent1_y = copy(sol1.y)
    parent2_z = copy(sol2.z)
    parent2_y = copy(sol2.y)
    #println(parent1_z)
    #println(parent2_z)
    list_pts = sort(randperm(t-1)[1:nbr_pts] .+ 1)
    #println(list_pts)
    z1 = cross(parent1_z, parent2_z, list_pts, nbr_pts, t)
    c1 = zeros(Float64,t)
    for j in 1:t
        if z1[j] ==1
            c1[j] = cmax
        else
            c1[j] = alpha*c1[j-1]
        end
    end
    #println(z1)
    z2 = cross(parent2_z, parent1_z, list_pts, nbr_pts, t)
    c2 = zeros(Float64,t)
    for j in 1:t
        if z2[j] ==1
            c2[j] = cmax
        else
            c2[j] = alpha*c2[j-1]
        end
    end
    #println(z2)
    y1 = zeros(Int,p,t)
    y2 = zeros(Int64,p,t)
    #display(parent1_y)
    #display(parent2_y)
    list_pts = sort(randperm(t-1)[1:nbr_pts] .+ 1)
    
    for i in P
        #println(list_pts)
        y1[i,:] = cross(parent1_y[i,:], parent2_y[i,:], list_pts, nbr_pts, t)
        y2[i,:] = cross(parent2_y[i,:], parent1_y[i,:], list_pts, nbr_pts, t) 
    end 
    
    #display(y1)
    #display(y2)
    
    unf_cost = sum(variable_prod_cost[i,j]*demand[i,j] + holding_cost[i,j]*demand[i,j] + set_up_cost[i,j] for i in P, j in T) + sum(mtn_cost[j] for j in T)
    
    feasible, x, I, obj  = resolve_CLSP(y1, c1, instance_dict)
    #println(feasible)
    if feasible
        obj += sum(set_up_cost[i,j]*y1[i,j] for i in P, j in T) + sum(mtn_cost[j]*z1[j] for j in T)
        fils1 = solution(x,I,y1,z1,c1,obj,0)
    else
        x_I = zeros(Float64, p,t)
        fils1 = solution(x_I,x_I,y2,z2,c2,unf_cost,0)
    end
        
    feasible, x, I, obj = resolve_CLSP(y2, c2, instance_dict)
    #println(feasible)
    if feasible
        obj += sum(set_up_cost[i,j]*y2[i,j] for i in P, j in T) + sum(mtn_cost[j]*z2[j] for j in T)
        fils2 = solution(x,I,y2,z2,c2,obj,0)
    else
        x_I = zeros(Float64, p,t)
        fils2 = solution(x_I,x_I,y2,z2,c2,unf_cost,0) 
    end 
    
    return fils1, fils2
end 





LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[1]:37

In [7]:
function find_index(T,z)
    indices_zeros, indices_ones = [], []
    for i in 2:t
        if z[i] == 1
            push!(indices_ones, i)
        else 
            push!(indices_zeros, i)
        end
    end
    indices_ones = shuffle(indices_ones)
    indices_zeros = shuffle(indices_zeros)
    
    return indices_zeros, indices_ones
end 

function mutation(sol, instance_dict, lag_step, nbr_ones_z, nbr_ones_y)
    T = instance_dict["T"]
    P = instance_dict["P"]
    t = length(T)
    p = length(P)
    cmax = instance_dict["cmax"]
    alpha = instance_dict["alpha"]
    mtn_cost = instance_dict["mtn_cost"]
    demand = instance_dict["demand"]
    set_up_cost = instance_dict["set_up_cost"]
    variable_prod_cost = instance_dict["variable_prod_cost"]
    holding_cost = instance_dict["holding_cost"]
    unf_cost = sum(variable_prod_cost[i,j]*demand[i,j] + holding_cost[i,j]*demand[i,j] + set_up_cost[i,j] for i in P, j in T) 
    + sum(mtn_cost[j] for j in T)
    #ind = rand(2:t)
    z_new1 = copy(sol.z)
    z_new2 = copy(sol.z)
    
    y_new1 = copy(sol.y)
    y_new2 = copy(sol.y)
    
    #println(z_new1)
    #display(y_new1)
    
    #indices_ones = [i for i in T if 2+lag_step <= i <= t-lag_step && sol.z[i] == 1]
    
    indices_zeros, indices_ones = find_index(t,sol.z)
    
    ind0_1 = indices_zeros[1]
    ind0_2 = indices_zeros[2]
    
    ind1_1 = indices_ones[1]
    ind1_2 = indices_ones[2]
    
    temp = sum(sol.z)
    #if temp >= nbr_ones_z
    z_new1[ind1_1] = 0
    z_new2[ind1_2] = 0
    #=
    else
        z_new1[ind1_1] = 0
        z_new1[ind0_1] = 1
        z_new2[ind1_2] = 0
        z_new2[ind0_2] = 1
    =#
    end
    println(sum(z_new1))
    println(sum(z_new2))
        
    c_new1 = construct_capacities(z_new1, t, alpha, cmax)
    c_new2 = construct_capacities(z_new2, t, alpha, cmax)
    
    
    
    #println(z_new1)
    ind = rand(2+lag_step:t-lag_step)
    for item in P
        
        indices_zeros, indices_ones = find_index(t,sol.y[item,:])
        #ind0_1 = indices_zeros[1]
        #ind0_2 = indices_zeros[2]
    
        ind1_1 = indices_ones[1]
        ind1_2 = indices_ones[2]
        temp = sum(sol.y[item,:])
        if temp >= nbr_ones_y[item]
            y_new1[item,ind1_1] = 0 
            y_new2[item,ind1_2] = 0 
            
        #=else
            y_new1[item,ind1_1] = 0
            y_new1[item,ind0_1] = 1
            y_new2[item,ind1_2] = 0 
            y_new2[item,ind0_2] = 1 
        =#
        end 
    end 
    #display(y_new1)
    
    feasible, x, I, obj = resolve_CLSP(y_new1, c_new1, instance_dict)
    #println(feasible)
    if feasible
        obj += sum(set_up_cost[i,j]*y_new1[i,j] for i in P, j in T) + sum(mtn_cost[j]*z_new1[j] for j in T)
        sol_new1 = solution(x,I,y_new1,z_new1,c_new1,obj,0)
    else
        x_I = zeros(Float64,p,t)
        sol_new1 = solution(x_I,x_I,y_new1,z_new1,c_new1,unf_cost,0) 
    end 
    
    feasible, x, I, obj = resolve_CLSP(y_new2, c_new2, instance_dict)
    #println(feasible)
    if feasible
        obj += sum(set_up_cost[i,j]*y_new2[i,j] for i in P, j in T) + sum(mtn_cost[j]*z_new2[j] for j in T)
        sol_new2 = solution(x,I,y_new1,z_new1,c_new1,obj,0)
    else
        x_I = zeros(Float64,p,t)
        sol_new2 = solution(x_I,x_I,y_new2,z_new2,c_new2,unf_cost,0) 
    end 
    
    return sol_new1, sol_new2
end 
    

LoadError: UndefVarError: z_new1 not defined

In [8]:
function copy_solution(sol)
    return solution(sol.x,sol.I,sol.y,sol.z,sol.c, sol.obj,sol.fitness)
end 

function genetic_algorithm(instance_dict, len_pop, nbr_iteration,)
    P = instance_dict["P"] 
    list_pop = pop_initial(len_pop,instance_dict)
    best_sol = list_pop[1]
    objectives = []
    objectives_bis = []
    proba = rand()
    compt1 = 0
    for iter in 1:nbr_iteration
        println("--------------------------Itération ", iter, " ----------------------------")
        #println("COMPT1 = ", compt1)
        list_pop = sort(list_pop, by = x -> x.obj)
        if list_pop[1].obj < best_sol.obj
            compt1 = 0
            best_sol = copy_solution(list_pop[1])
        else 
            compt1 += 1 
        end
        push!(objectives, best_sol.obj)
        push!(objectives_bis, list_pop[3].obj)
        nbr_ones_z = sum(best_sol.z)
        nbr_ones_y = []
        for i in P
            push!(nbr_ones_y, sum(best_sol.y[i,:]))
        end
        println("nbr_ones_z = ", nbr_ones_z)
        println("nbr_ones_y = ", nbr_ones_y)
            
        new_pop = []
        push!(new_pop, copy_solution(list_pop[1]))
        push!(new_pop, copy_solution(list_pop[2]))
        compt = 2

        println("CROSSOVER")
        nbr_crossover = div(len_pop,3)
        while compt < nbr_crossover
            sol1,sol2 =  crossover(copy_solution(list_pop[1]), copy_solution(list_pop[2]), 2, instance_dict);
            push!(new_pop, sol1);
            push!(new_pop, sol2);

            sol1,sol2 =  crossover(copy_solution(list_pop[1]), copy_solution(list_pop[3]), 2, instance_dict);
            push!(new_pop, sol1);
            push!(new_pop, sol2);

            sol1,sol2 =  crossover(copy_solution(list_pop[2]), copy_solution(list_pop[3]), 2, instance_dict);
            push!(new_pop, sol1);
            push!(new_pop, sol2);
            compt += 6 
        end

        println("MUTATION")
        len_new_pop = compt
        while compt < len_pop
            i = rand(1:len_new_pop)
            sol1, sol2 = mutation(copy_solution(new_pop[i]),instance_dict, 1, nbr_ones_z, nbr_ones_y);
            push!(new_pop, sol1)
            push!(new_pop, sol2)

            i = rand(1:len_new_pop)
            sol1, sol2 = mutation(copy_solution(new_pop[i]),instance_dict, 2, nbr_ones_z, nbr_ones_y);
            push!(new_pop, sol1)
            push!(new_pop, sol2)


            compt +=4
        end
            #new_pop = sort(new_pop, by = x -> x.obj)  
        list_pop = new_pop;
        println(best_sol.obj)
    end 
    return objectives, objectives_bis
end

genetic_algorithm (generic function with 1 method)

In [9]:
p = 30
t = 35
file_path = "instances/rd_instance" * string(p) * "_" * string(t) * ".txt";
instance_dict = init.gen_instance(p,t, fp=file_path); 
instance_dict["P"] = 1:p;
instance_dict["T"] = 1:t;

Set of periods in the planning horizon T: 
35
Set of products P: 
30
Capacity reduction coefficient alpha: 
0.74
Relative decrease of capacity per set up gamma: 
0.37
Capacity reduction coefficient due to product types beta: 
[0.26, 0.25, 0.32, 0.21, 0.3, 0.3, 0.21, 0.39, 0.31, 0.34, 0.28, 0.35, 0.21, 0.26, 0.31, 0.25, 0.34, 0.4, 0.25, 0.35, 0.26, 0.28, 0.4, 0.27, 0.22, 0.37, 0.32, 0.32, 0.31, 0.35]
Fixed production cost of each product in each period f: 
[[1664, 1574, 761, 1313, 1004, 1479, 945, 755, 1514, 983, 774, 829, 1661, 487, 368, 1676, 1323, 682, 1280, 761, 1730, 1307, 854, 1300, 979, 1005, 556, 1110, 1730, 975, 1378, 1482, 588, 1137, 367], [1111, 533, 452, 1040, 891, 445, 527, 1544, 1137, 1294, 513, 463, 1429, 495, 607, 982, 714, 1416, 413, 1269, 1701, 1613, 1377, 1738, 608, 1017, 1724, 1532, 1370, 954, 1458, 1523, 1555, 1535, 1386], [1075, 364, 448, 1744, 1677, 603, 1680, 1519, 1707, 1225, 1634, 481, 643, 869, 1365, 1729, 434, 1546, 1168, 1420, 1195, 417, 1712, 1194, 1725, 65

In [10]:
instance_dict["P"]

1:30

In [11]:
len_pop = 30;
nbr_iteration = 20
@time begin
    objectives, objectives_bis = genetic_algorithm(instance_dict, len_pop, nbr_iteration);
end

1:30
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Set parameter Username
Academic license - for non-commercial use only - expires 2024-

LoadError: BoundsError: attempt to access Int64 at index [2]

In [1]:
a = plot(1:nbr_iteration, objectives)
b = plot(1:nbr_iteration, objectives_bis)
display(a)
display(b)

LoadError: UndefVarError: nbr_iteration not defined

In [13]:
println(objectives)

LoadError: UndefVarError: objectives not defined

In [14]:
sz, sc, sy, sx, sI = model(instance_dict);


Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-10
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2204 rows, 3220 columns and 6442 nonzeros
Model fingerprint: 0xc2165bea
Variable types: 2135 continuous, 1085 integer (1085 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+04]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+04]
Found heuristic solution: objective 1.170391e+07
Presolve removed 127 rows and 124 columns
Presolve time: 0.01s
Presolved: 2077 rows, 3096 columns, 6191 nonzeros
Variable types: 2041 continuous, 1055 integer (1054 binary)
Found heuristic solution: objective 1.159348e+07

Root relaxation: objective 7.228331e+06, 2658 iterations, 0.01 seconds (0.00 work units)

    Nodes  

In [15]:
sum(sz)

13.0

In [16]:
#Fonction pour générer la population initiale 
function generate_pop_initial(len_pop,instance_dict, unf_obj)
    P = instance_dict["P"]
    T = instance_dict["T"]
    p = length(P)
    t = length(T)
    alpha = instance_dict["alpha"]
    cmax = instance_dict["cmax"]
    mtn_cost = instance_dict["mtn_cost"]
    set_up_cost = instance_dict["set_up_cost"]
    
    pop_initial_list = []
    
    nbr_mtn = t
    compt = 0
    feasibility = true
    while feasibility && compt <= len_pop
        #println("------------------", compt, "--------------------")
        z = vcat(ones(Int64, nbr_mtn-1), zeros(Int64, t-nbr_mtn+1))
        shuffle!(z)
        z[1] = 1
        nbr_mtn -= 1
        #println(z)
        
        c = construct_capacities(z, t, alpha, cmax)
        
        y = zeros(Int64, p,t)
        for item in P
            line = vcat(ones(Int64, t-compt), zeros(Int64, compt))
            shuffle!(line)
            y[item, :] = line
        end
        y[:,1] = [1 for _ in P]
        #display(y)
        
        clsp_sol = resolve_CLSP(y,c,instance_dict)
        feasibility, x, I, clsp_obj = clsp_sol["feasibility"], clsp_sol["x"], clsp_sol["I"], clsp_sol["clsp_obj"]
        if feasibility
            sol_obj = clsp_obj + sum(set_up_cost .* y) + dot(mtn_cost,z)
            push!(pop_initial_list, solution(x,I,y,z,c,sol_obj,0,feasibility));
            compt += 1
        end
       
        #println(feasibility)
    end 
    #=
    println()
    println(nbr_mtn)
    println(compt)
    println()
    =#
    nbr_mtn += 1
    feasibility = true
    compt_y = 1
    while feasibility && compt < len_pop
        #println("------------------", compt, "--------------------")
    
        z = vcat(ones(Int64, nbr_mtn-1), zeros(Int64, t-nbr_mtn+1))
        shuffle!(z)
        z[1] = 1
        nbr_mtn -= 1
        #println(z)
        
        c = construct_capacities(z, t, alpha, cmax)
        
        y = vcat(ones(Int64, p*t-compt_y), zeros(Int64, compt_y))
        shuffle!(y)
        y = reshape(y, p, t)
        y[:,1] = [1 for _ in P]
        compt_y += 1 
        #display(y)
        clsp_sol = resolve_CLSP(y,c,instance_dict)
        feasibility, x, I, clsp_obj = clsp_sol["feasibility"], clsp_sol["x"], clsp_sol["I"], clsp_sol["clsp_obj"]
        if feasibility
            sol_obj = clsp_obj + sum(set_up_cost .* y) + dot(mtn_cost,z)
            push!(pop_initial_list, solution(x,I,y,z,c,sol_obj,0,feasibility));
            compt += 1
        end
        
    end
    #=
    println()
    println(nbr_mtn)
    println(compt)
    println()
    =#
    nbr_mtn += 1
    feasibility = true
    compt_y = 1
    while compt < len_pop
        #println("------------------", compt, "--------------------")
    
        z = vcat(ones(Int64, nbr_mtn), zeros(Int64, t-nbr_mtn))
        shuffle!(z)
        z[1] = 1
        #println(z)
        c = construct_capacities(z, t, alpha, cmax)
        y = vcat(ones(Int64, p*t-compt_y), zeros(Int64, compt_y))
        shuffle!(y)
        y = reshape(y, p, t)
        y[:,1] = [1 for _ in P]
        compt_y += 1 
        #display(y)
        mtn_set_up_costs = sum(set_up_cost .* y) + dot(mtn_cost,z)
        clsp_sol = resolve_CLSP(y,c,instance_dict)
        feasibility, x, I, clsp_obj = clsp_sol["feasibility"], clsp_sol["x"], clsp_sol["I"], clsp_sol["clsp_obj"]
        if feasibility
            sol_obj = clsp_obj + mtn_set_up_costs
            push!(pop_initial_list, solution(x,I,y,z,c,sol_obj,0,feasibility));
            compt += 1
        else
            x = zeros(Float64, p, t)
            sol_obj = unf_obj + mtn_set_up_costs
            push!(pop_initial_list, solution(x,x,y,z,c,sol_obj,0,feasibility));
            compt += 1
        end
        
    end
    
    #println(length(pop_initial_list))
    #println(nbr_mtn)
    return pop_initial_list

end 


LoadError: UndefVarError: obj not defined

6

3-element Vector{Int64}:
 1
 3
 2

10-element Vector{Int64}:
 10
  1
  8
  3
  7
  6
  5
  9
  2
  4